<a href="https://colab.research.google.com/github/ktheu/KursNotebooks/blob/master/MazeColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls gdrive/'My Drive'

In [0]:
from google.colab import files
uploaded = files.upload()

In [9]:
f = open("maze02.txt")       
lines = f.read().splitlines()
grid = [list(s) for s in lines]

def showGrid():
    for a in grid:
        print(''.join(a))
        
showGrid()

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                                                                 X
X                                                                 X
X                                                                 X
X                       XXXXXXXXXXXXXXXXX                         X
X                       X                                         X
X                       X                     X                   X
X                       X                     X                   X
X                       X                     X                   X
X                       X                              X          X
X                       X      XXXXXXXXXXXXXXXXXXXX    X          X
X                       X                              X          X
X                       X           E                  X          X
X                       XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX          X
X                                               

In [0]:
from collections import deque
from heapq import heappop, heappush
'''
Ein state ist ein Tupel (x,y) in einem grid (einer
Liste mit Listen von Tupeln)
'''
        
def getStart():
    for x in range(len(grid)):
        for y in range(len(grid[x])):
            if grid[x][y] == 'S':
                return x, y
def getGoal():
    for x in range(len(grid)):
        for y in range(len(grid[x])):
            if grid[x][y] == 'E':
                return x, y

def goaltest(state):
    return grid[state[0]][state[1]]  == 'E'

def nextstates(state):
    x, y = state
    tmp = []
    for xd, yd in dirs:
        xn, yn = x+xd, y+yd
        if grid[xn][yn] != 'X':
            tmp.append((xn,yn))
    return tmp

def reconstructPath(prev,state):
    s = state
    tmp = []
    while prev[s] is not None:
        tmp.append(s)
        s = prev[s]
    tmp.append(s)
    tmp.reverse()
    return tmp 

def bfs(s):
    frontier =  deque([s])
    prev = {s:None}
    while frontier:
        state = frontier.popleft()
        if goaltest(state):
            return prev, state, frontier
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state

def dfs(s):
    frontier =  [s]
    prev = {s:None}
    
    while frontier:
        state = frontier.pop()  
        if goaltest(state):
            return prev, state, frontier
        nxt = nextstates(state)
        nxt.reverse()        # die linken Kinder sollen zuletzt auf den frontier-Keller
        for v in nxt:
            if v not in prev:
                frontier.append(v)
                prev[v] = state

def h(state):
    ''' Manhatten-Distanz '''
    x1, y1 = state
    x2, y2 = goalstate
    return abs(x1-x2) + abs(y1-y2)

def h2(state):
    ''' Euklidsche-Distanz '''
    x1, y1 = state
    x2, y2 = goalstate
    return ((x1-x2)**2 + (y1-y2)**2)**0.5   

def greedy(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    while frontier:
        _ ,state = heappop(frontier)  
        if goaltest(state):
            return prev,frontier
        for v in nextstates(state):
            if v not in prev:
                heappush(frontier,(h(v),v))
                prev[v] = state

def astar(s):
    frontier =[(h(s),s)]  
    prev = {s:None}
    g = {s:0}                         # backword costs 
    while frontier:
        _ ,state = heappop(frontier)  # die Kosten braucht man an der Stelle nicht
        if goaltest(state):
            return prev,frontier
        for v in nextstates(state):
            gg = g[state] + 1
            if v not in prev or gg < g[v]:
                g[v] = gg
                heappush(frontier,(g[v]+h(v),v))
                prev[v] = state
                
def showResult():  
    # für bfs und dfs
    for x,y in prev:
        grid[x][y]='.'
    for x,y in path:
        grid[x][y] = 'o'
    for x,y in frontier:
        grid[x][y] = '~'
    x,y = startstate
    grid[x][y] = 'S'
    x,y = goalstate
    grid[x][y] = 'E'
    showGrid()
 
    print("explored = {}, weglänge = {}".format(len(prev)-len(frontier), len(path)-1))
    print("gefundener Weg: 'o', explored: '.', frontier: '~'")
    
def showResult2():
    # für greedy und astar 
    for x,y in prev:
        grid[x][y]='.'
    for x,y in path:
        grid[x][y] = 'o'
    for tup in frontier:
        _, state = tup
        x, y = state
        grid[x][y] = '~'
    x,y = startstate
    grid[x][y] = 'S'
    x,y = goalstate
    grid[x][y] = 'E'
    showGrid()
 
    print("explored = {}, weglänge = {}".format(len(prev)-len(frontier), len(path)-1))
    print("gefundener Weg: 'o', explored: '.', frontier: '~'")
    
N, E, S, W  = (-1,0), (0,1), (1,0), (0,-1)
dirs = [N,S,W,E]

#### Breitensuche

In [11]:
f = open("maze02.txt")       
lines = f.read().splitlines()
grid = [list(s) for s in lines]

startstate = getStart()
prev, goalstate, frontier = bfs(startstate)     # bfs, dfs, greedy, astar
path = reconstructPath(prev,goalstate)

showResult()

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X.................................................................X
X.................................................................X
X.....oooooooooooooooooooooooooooooooooooo........................X
X.....o.................XXXXXXXXXXXXXXXXXo........................X
X.....o.................X................o........................X
X.....o.................X................o....X...................X
X.....o.................X................o....X...................X
X.....o.................X................o....X...................X
X.....o.................X.....oooooooooooo.............X..........X
X.....o.................X.....oXXXXXXXXXXXXXXXXXXXX....X..........X
X.....o.................X.....o......~     ~...........X..........X
X.....o.................X.....ooooooE       ~..........X..........X
X.....o.................XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX..........X
X.....o.........................................

#### Tiefensuche


In [37]:
f = open("gdrive/My Drive/data/maze01.txt")    
lines = f.read().splitlines()
f.close()
grid = [list(s) for s in lines]

startstate = getStart()
prev, goalstate, frontier = dfs(startstate)     # bfs, dfs, greedy, astar
path = reconstructPath(prev,goalstate)

showResult()

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X..........~ooo~ooo~ooo~ooo~ooo~ooo~ooo~                          X
X..........~o~o~o~o~o~o~o~o~o~o~o~o~o~o~                          X
X..........~o~o~o~o~o~o~o~o~o~o~o~o~o~o~                          X
X..........~o~o~o~o~o~o~o~o~o~o~o~o~o~o~                          X
X..........~o~o~o~o~o~o~o~o~o~o~o~o~o~o~                          X
X..........~o~o~o~o~o~o~o~ooo~ooo~ooo~o~                          X
X..........~o~o~o~o~o~o~o~XXXXXXXXXXXXo~                          X
X..........~o~o~o~o~o~o~o~X        Eo~o~                          X
X..........~o~o~o~o~o~o~o~X        ~o~o~                          X
X..........~o~o~o~o~o~o~o~X        ~o~o~                          X
X..........~o~S~o~o~o~o~o~X        ~o~o~                          X
X..........~o~~oo~o~o~o~o~X        ~o~o~                          X
X..........~o~oo~~o~o~o~o~         ~o~o~                          X
X..........~o~o~.~o~o~o~o~         ~o~o~        

#### Greedy

In [13]:
f = open("maze02.txt")       
lines = f.read().splitlines()
grid = [list(s) for s in lines]

startstate = getStart()
goalstart = getGoal()
prev, frontier = greedy(startstate)     # bfs, dfs, greedy, astar
path = reconstructPath(prev,goalstate)

showResult2()

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                               ~.......~                         X
X                      ~~~~~~~~~.........~                        X
X                     ~ooooooooooooooooooo~                       X
X                     ~oXXXXXXXXXXXXXXXXXo~                       X
X                    ~.oX        ~..oooooo~                       X
X                   ~..oX       ~...o...~~    X                   X
X                  ~...oX      ~....o....~    X                   X
X                 ~....oX     ~.....o.....~   X                   X
X                ~.....oX    ~ooooooo.....~            X          X
X               ~......oX    ~oXXXXXXXXXXXXXXXXXXXX    X          X
X     ~~~~~~~~~~.......oX    ~ooooooo~                 X          X
X    ~ooooooooooooooooooX     ~~~~~~E                  X          X
X    ~o~~~~~~~~~........XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX          X
X    ~o~        ~...............................

#### A*

In [36]:
f = open("gdrive/My Drive/data/maze01.txt")       
lines = f.read().splitlines()
f.close()
grid = [list(s) for s in lines]

startstate = getStart()
goalstate = getGoal()
prev, frontier = astar(startstate)     # bfs, dfs, greedy, astar
path = reconstructPath(prev,goalstate)

showResult2()

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X                                                                 X
X                                                                 X
X                                                                 X
X                                                                 X
X             ~~~~~~~~~~~~~~~~~~~~~~                              X
X            ~......................~                             X
X           ~.............XXXXXXXXXXXX                            X
X          ~..............XooooooooE                              X
X          ~..............Xo~~~~~~~                               X
X          ~..............Xo~                                     X
X          ~..SoooooooooooXo~                                     X
X           ~............oXo~                                     X
X            ~...........ooo~                                     X
X             ~~~~~~~~~~~~~~                    